In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import pandas as pd, numpy as np 
from travis_attack.config import Config
from travis_attack.insights import get_training_dfs, _prepare_df_concat
from travis_attack.utils import display_all
from IPython.core.debugger import set_trace
import logging 
logger = logging.getLogger("run")

In [ ]:
cfg = Config()
#run_id = 'cdy5d5yu'
run_name = "imperial-nexu-132"
path_run = f"{cfg.path_checkpoints}{run_name}/"

## Looking at some examples 

Below we just look at some examples to get a feel for what is going on. 

In [ ]:
def get_interesting_idx(df, n):
    def get_idx_with_top_column_values(cname, n=5, ascending=False):
        return df[['idx',cname]].\
            drop_duplicates().\
            sort_values(cname, ascending=ascending)\
            ['idx'][0:n].values.tolist()
    
    def sample_idx_with_label_flips(n=5): 
        df1 = df[['idx','label_flip']].query("label_flip!=0")
        if len(df1) == 0 : print("No label flips detected"); return None
        else: return df1.drop_duplicates()['idx'].sample(n).values.tolist()
    
    idx_d = dict(
        random = df.idx.drop_duplicates().sample(n).tolist(),
        label_flips = sample_idx_with_label_flips(n=n),
        idx_n_unique_pp  = get_idx_with_top_column_values('idx_n_unique_pp',n=n,ascending=False),
       # idx_n_pp_changes = get_idx_with_top_column_values('idx_n_pp_changes',n=n,ascending=False),
        high_contradiction = get_idx_with_top_column_values('contradiction_score',n=n,ascending=False)
    )
    return idx_d

def print_stats(df, idx_d, key, i):
    print("\n###############\n")
    print(key, i, "\n")
    if idx_d[key] is None: return
    idx = idx_d[key][i]
    # Setup 
    df1 = df.query('idx==@idx')
    orig = pd.unique(df1['orig_l'])[0]
    print("Original:", orig)
    print("Original label", pd.unique(df1['orig_label'])[0] )
    pp_all = list(df1['pp_l'])
    #print("All paraphrases", pp_all)
    pp_unique = list(pd.unique(df1['pp_l']))
    n_pp_unique = len(pp_unique)

    # showing a "timeline" of how the paraphrases change over the epochs
    g_fields = ["pp_l","pp_truelabel_probs","vm_score","sts_score","pp_letter_diff", "contradiction_score", "reward","label_flip", 'pp_logp',
       'ref_logp', 'kl_div', 'reward_with_kl', 'loss']
    #g_fields = ["pp_l","vm_score"]
    g = df1.groupby(g_fields).agg({'epoch' : lambda x: list(x)})
    g = g.sort_values(by='kl_div', key = lambda col: col.map(lambda x: np.min(x)))
    print("Unique paraphrases:", n_pp_unique)
    print("How the paraphrases change:")
    display_all(g)

    # Showing a dataframe of the few best paraphrases
    best_pps = df1.sort_values('reward', ascending=False).iloc[0]
    print("Best Paraphrase")
    display_all(best_pps.to_frame().T)
        
def print_interesting_text_stats(df, n): 
    idx_d = get_interesting_idx(df, n)
    for key in idx_d.keys():
        for i in range(n): 
            print_stats(df, idx_d, key,i)

In [ ]:
split = 'valid'
df_d = get_training_dfs(path_run, postprocessed=True)
idx_d = get_interesting_idx(df_d[split], n=4)
print_interesting_text_stats(df_d[split], n=4)


###############

random 0 

Original: it's more enjoyable than i expected , though , and that's because the laughs come from fairly basic comedic constructs . cinematic pratfalls given a working over . the cast is spot on and the mood is laid back .
Original label 1
Unique paraphrases: 2
How the paraphrases change:


,,,,,,,,,,,,,epoch
pp_l,pp_truelabel_probs,vm_score,sts_score,pp_letter_diff,contradiction_score,reward,label_flip,pp_logp,ref_logp,kl_div,reward_with_kl,loss,
the laughs are based on fairly basic comedic constructs the comedy characters are being prepared for a meeting the cast is very good and the mood is laid,0.90352,-0.26434,0.81202,58,0.013632,0.0,0,-1.380464,-1.947837,0.567372,0.0,0.0,[1]
laughs come out of basic comedy constructs cinematic pratfalls are worked on film cast the mood is laid out well,0.87442,-0.23524,0.84009,99,0.006258,0.0,0,-1.013404,-3.026644,2.013240,0.0,0.0,[2]


Best Paraphrase


,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,contradiction_score,reward,pp_logp,ref_logp,kl_div,reward_with_kl,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
42,24,1,"it's more enjoyable than i expected , though , and that's because the laughs come from fairly basic comedic constructs . cinematic pratfalls given a working over . the cast is spot on and the mood is laid back .",the laughs are based on fairly basic comedic constructs the comedy characters are being prepared for a meeting the cast is very good and the mood is laid,0.639178,0.90352,0.903518,1,1,0,-0.26434,0.81202,58,0.725118,0.0136321,0,-1.38046,-1.94784,0.567372,0,0,3,931,1,0,0,0.125,64,16,51,16,2,1,1,37,1,7,0,165,28,1,0,0,126,9,0,7,0,39,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, 25, 26, 27, 28, 38, 39, 40, 44, 45, 53, 54]","[it 's more enjoyable than i expected, though, and that 's because, come from, . cinematic pratfalls given, working over., spot on, back.]","[18, 19, 20, 29, 30, 31, 32, 33, 34, 35, 37, 46, 47]","[are based on, the comedy characters are being prepared for, meeting, very good]","[14, 15, 21, 22, 23, 24, 36, 41, 42, 43, 48, 49, 50, 51, 52]","[the laughs, fairly basic comedic constructs, a, the cast is, and the mood is laid]",4,6,13,27,True,False,False,30



###############

random 1 

Original: it may scream low budget , but this charmer has a spirit that cannot be denied .
Original label 1
Unique paraphrases: 2
How the paraphrases change:


...

,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,contradiction_score,reward,pp_logp,ref_logp,kl_div,reward_with_kl,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
1417,839,2,"ill-considered , unholy hokum .",hokum is unkind,0.933523,0.76449,0.764492,0,0,0,0.16903,0.8457,16,0.483871,0.563946,2.02837,-1.30718,-2.91009,1.60291,1.86808,2.44192,46,1862,0,18.374,inf,0.3125,24,16,15,16,2,1,1,3,1,3,0,24,3,1,0,0,13,0,0,3,0,11,"[0, 1, 2, 3, 4, 6]","[ill- considered, unholy, .]","[7, 8]",[is unkind],[5],[hokum],1,2,2,6,True,False,False,7



###############

high_contradiction 3 

Original: god bless crudup and his aversion to taking the easy hollywood road and cashing in on his movie-star gorgeousness .
Original label 1
Unique paraphrases: 2
How the paraphrases change:


,,,,,,,,,,,,,epoch
pp_l,pp_truelabel_probs,vm_score,sts_score,pp_letter_diff,contradiction_score,reward,label_flip,pp_logp,ref_logp,kl_div,reward_with_kl,loss,
god bless crudup for his aversion to the hollywood road and his stunningness,0.93969,-0.01340,0.87819,39,0.009787,0.0,0,-0.920229,-1.516751,0.596522,0.0,0.0,[1]
pity the crudup and his love for the films,0.23454,0.69175,0.73584,73,0.987051,0.0,1,-2.078466,-2.869893,0.791427,0.0,0.0,[2]


Best Paraphrase


,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,contradiction_score,reward,pp_logp,ref_logp,kl_div,reward_with_kl,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
474,274,1,god bless crudup and his aversion to taking the easy hollywood road and cashing in on his movie-star gorgeousness .,god bless crudup for his aversion to the hollywood road and his stunningness,0.926291,0.93969,0.939687,1,1,0,-0.0134,0.87819,39,0.66087,0.00978705,0,-0.920229,-1.51675,0.596522,0,0,26,931,1,8.6131,0.783009,0.25,40,16,33,16,2,1,2,19,1,2,0,94,13,1,0,0,64,6,0,2,0,30,"[3, 8, 10, 14, 15, 16, 19, 20, 21, 22, 23]","[and, taking, easy, cashing in on, movie- star gorgeousness.]","[4, 18]","[for, stunningness]","[0, 1, 2, 5, 6, 7, 9, 11, 12, 13, 17]","[god bless crudup, his aversion to, the, hollywood road and, his]",2,5,2,11,True,False,False,11


In [ ]:
df_d['training_step']

,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,...,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
0,0,1,the rock is destined to be the 21st century's ...,The rock is going to be the new conan and he w...,0.855556,0.27051,0.729486,1,0,1,...,"[2, 3, 6, 7, 8, 12, 14, 16, 18, 41, 49, 50, 51...","[rock is, to be the, new, conan, and, he, a, s...",5,10,22,19,False,True,False,28
1,0,2,the rock is destined to be the 21st century's ...,The rock is going to be the new conan with its...,0.855556,0.84938,0.849378,1,1,0,...,"[2, 3, 6, 7, 8, 12, 14, 24, 34, 35, 36, 37, 38...","[rock is, to be the, new, conan, to, arnold sc...",4,6,8,18,False,True,False,18
2,0,3,the rock is destined to be the 21st century's ...,The rock is going to make a splash at the beac...,0.855556,0.66735,0.667351,1,1,0,...,"[2, 3, 19, 20, 21, 22, 23, 47, 61]","[rock is, going to make a splash, ,, .]",3,4,24,29,False,True,False,32
3,0,4,the rock is destined to be the 21st century's ...,The new 21 sauncies's conan is a rock and it's...,0.855556,0.71693,0.716926,1,1,0,...,"[9, 22, 26, 27, 36, 50, 51, 55, 56, 60]","[rock, and, 's going, than, schwarzenegger,, v...",5,8,24,28,True,False,False,33
4,0,5,the rock is destined to be the 21st century's ...,The conan is the new rock for 21st century as ...,0.855556,0.80401,0.804009,1,1,0,...,"[3, 5, 12, 13, 36, 50, 54, 55]","[the, rock, 21st century, he, jean, van damme]",8,5,20,30,True,False,False,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37210,8529,1,"things really get weird , though not particula...","The movie is not terribly frightening, in fact...",0.944194,0.75195,0.751954,0,0,0,...,"[12, 13, 25, 30]","[movie is, and, .]",3,3,12,15,False,False,False,16
37211,8529,2,"things really get weird , though not particula...",The movie gets very weird but not scary as it ...,0.944194,0.66512,0.665115,0,0,0,...,"[8, 12, 14, 20, 29]","[weird, not, scary, is, .]",6,6,10,14,False,False,False,15
37212,8529,3,"things really get weird , though not particula...",Though not particularly frightening the only m...,0.944194,0.93465,0.934647,0,0,0,...,"[9, 10, 14, 25, 26, 27, 28, 29]","[not particularly, the, portent and no content.]",5,4,8,11,False,False,False,13
37213,8529,4,"things really get weird , though not particula...",The movie in question had no movie clips or mo...,0.944194,0.91285,0.912847,0,0,0,...,"[12, 24, 25]","[movie, and no]",3,3,11,16,True,False,False,16


## Looking at common removals and insertions 

In [ ]:
def get_common_removals_and_insertions(df_concat): 
    idx = df_concat[['data_split','orig_l', 'pp_l']].drop_duplicates().index
    df_unique_pp = df_concat[['data_split','orig_l', 'pp_l','insertions', 'removals']].iloc[idx]
    def flatten_list(l): return [item for sublist in l for item in sublist] 
    removals_flat   =  flatten_list(df_unique_pp['removals'].values)
    insertions_flat =  flatten_list(df_unique_pp['insertions'].values)
    return pd.value_counts(removals_flat), pd.value_counts(insertions_flat)

In [ ]:
df_concat = _prepare_df_concat(df_d)
removals, insertions = get_common_removals_and_insertions(df_concat)

print("\n#### REMOVALS ####\n")
print(removals.head(30))
print("\n#### INSERTIONS ####\n")
print(insertions.head(30))


#### REMOVALS ####

.        17120
the       5858
,         4029
-         2214
a         1629
this      1433
's        1417
it        1269
its       1187
and       1168
is        1061
that       924
...        888
as         858
of         823
'          808
there      781
it 's      690
in         597
an         561
with       560
to         532
"          499
--         474
for        469
if         455
i          419
but        408
one        385
what       368
dtype: int64

#### INSERTIONS ####

The         11880
the          3635
A            3438
is           2245
a            2044
and          2011
was          1798
,            1532
that         1466
It           1310
There is     1273
of           1183
"            1178
it           1134
but          1125
There        1059
in           1053
about        1019
This          938
movie         871
film          857
has           830
with          818
not           707
In            674
are           595
If            589
they   

Here you can look at a specific phrase and examples of where it appears. 

In [ ]:
def investigate_phrase(phrase, cname, n ): 
    mask = [phrase in strs for strs in df_concat[cname]]
    display_all(df_concat[mask].sample(n))

In [ ]:
investigate_phrase('despite', 'removals', 4)

,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,contradiction_score,reward,pp_logp,ref_logp,kl_div,reward_with_kl,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_contradiction_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_ref_logprobs,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level,data_split
66692,6753,3,"despite a powerful portrayal by binoche , it's a period romance that suffers from an overly deliberate pace and uneven narrative momentum .",A period romance had an over deliberate pace and narrative momentum because of a portrayal by binoche,0.926852,0.81810,0.818103,0,0,0,0.10875,0.93341,38,0.726619,0.010365,0.00000,-0.793267,-7.818438,7.025171,0.000000,0.000000,191,5583,1,22.848427,3.264061,0.1875,32,16,30,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,4,2.0,22,1,3,0,114,17,1,0,0,85,5,0,3,0,29,"[0, 15, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]","[despite, powerful, , it 's a period romance that suffers from an overly deliberate pace and uneven narrative momentum.]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]",[A period romance had an over deliberate pace and narrative momentum because of],"[14, 16, 17, 18]","[a, portrayal by binoche]",1,3,13,20,True,False,False,21,eval_train
14727,3393,3,"a delightful surprise because despite all the backstage drama , this is a movie that tells stories that work -- is charming , is moving , is funny and looks professional .",is not only nice surprise because it is all about a movie except for it being about stories relating to the business of a movie.,0.956492,0.55920,0.559200,1,1,0,0.39729,0.77958,43,0.748538,0.582148,0.00000,-2.594275,-5.216480,2.622205,0.000000,0.000000,415,4137,1,4.578536,0.572317,0.0000,40,4,46,4,0.941276,1.805120,0.055612,0.008888,0.025492,0.000106,0.020336,0.015635,0.004094,0.007038,1.733341,0.000323,1.107938,0.040554,0.180623,0.103278,5,4,5.0,27,1,6,0,134,25,1,1,0,103,2,0,5,0,31,"[0, 1, 8, 13, 14, 15, 16, 17, 18, 21, 22, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]","[a delightful, despite, the backstage drama, this is, that tells, that work -- is charming, is moving, is funny and looks professional]","[2, 3, 4, 5, 9, 10, 12, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35]","[is not only nice, it is, about, except for it being about, relating to the business of a movie]","[6, 7, 11, 19, 20, 28, 50]","[surprise because, all, a movie, stories, .]",5,5,19,25,False,False,False,29,training_step
12490,2850,1,"a playful iranian parable about openness , particularly the need for people of diverse political perspectives to get along despite their ideological differences .","There is a need for people of different political perspectives to get along, in iran there is a playful parable about openness.",0.949001,0.90945,0.909451,1,1,0,0.03955,0.96166,35,0.783951,0.000162,0.00000,-0.812997,-5.402091,4.589094,0.000000,0.000000,781,781,1,4.002707,0.500338,0.2500,32,4,35,4,0.705688,1.409804,0.057427,0.009567,0.025834,0.000125,0.021050,0.013364,0.003378,0.006222,1.338448,0.000356,0.856105,0.038602,0.179789,0.103327,5,4,1.0,22,1,2,